# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [ ]:
!pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [367]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [368]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [373]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [374]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [375]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

In [370]:
from pprint import pprint

Let's take a look at some of the documents we've managed to split.

In [376]:
some_documents = split_documents[0:1]
pprint(some_documents)

['\ufeff\n'
 'The Pmarca Blog Archives\n'
 '(select posts from 2007-2009)\n'
 'Marc Andreessen\n'
 'copyright: Andreessen Horowitz\n'
 'cover design: Jessica Hagy\n'
 'produced using: Pressbooks\n'
 'Contents\n'
 'THE PMARCA GUIDE TO STARTUPS\n'
 'Part 1: Why not to do a startup 2\n'
 'Part 2: When the VCs say "no" 10\n'
 'Part 3: "But I don\'t know any VCs!" 18\n'
 'Part 4: The only thing that matters 25\n'
 'Part 5: The Moby Dick theory of big companies 33\n'
 'Part 6: How much funding is too little? Too much? 41\n'
 "Part 7: Why a startup's initial business plan doesn't\n"
 'matter that much\n'
 '49\n'
 'THE PMARCA GUIDE TO HIRING\n'
 'Part 8: Hiring, managing, promoting, and Dring\n'
 'executives\n'
 '54\n'
 'Part 9: How to hire a professional CEO 68\n'
 "How to hire the best people you've ever worked\n"
 'with\n'
 '69\n'
 'THE PMARCA GUIDE TO BIG COMPANIES\n'
 'Part 1: Turnaround! 82\n'
 'Part 2: Retaining great people 86\n'
 'THE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\n'
 'AND SOM

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [377]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

### Answer

1. Yes, through the optional, dimensions parameter.
2. Through Matryoshka Representation Learning.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [378]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

### Answer

It is more efficient and reduces latency. My understanding is it allows us to collect multiple embeddings aysnc instead of sequentially.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [25]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Answer

For variability in the language and 'speaking' style of the response, setting the seed parameter or setting temperature to 0 can help achieve more reproducible outputs. As well as setting the top_p and top_k to high values could help. For more reproducible response formats, we can specify that directly in the prompt - aka "please provide your response in the following JSON format: ..."

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [386]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [27]:
print(response)

The "best" way to write a loop in Python can depend on the specific use case and what you are trying to achieve. However, I can share some common practices and examples that make loops effective and easy to read. 

### Loop Basics

1. **Using `for` Loops**:
   The `for` loop is often used for iterating over a sequence (like a list, tuple, string, or range).

   ```python
   # Example: Iterating over a list
   fruits = ['apple', 'banana', 'cherry']
   for fruit in fruits:
       print(fruit)
   ```

2. **Using `while` Loops**:
   The `while` loop continues until a specified condition is false.

   ```python
   # Example: Counting down
   countdown = 5
   while countdown > 0:
       print(countdown)
       countdown -= 1
   ```

### Best Practices

- **Keep it Readable**:
  Use descriptive variable names and maintain proper indentation for clarity.

- **Use `enumerate()` for Indexing**:
  When you need both the index and value from a list, `enumerate()` is a great choice.

  ```python
  

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [209]:
RAG_PROMPT_TEMPLATE = """ \
You have been provided with the Forrest Gump movie script. Your job is to use the provided context to answer the user's query. Users may have questions around particular characters, famous quotes, or explanations of certain scenes. Please provide answers to the user's questions if they pertain to the context.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through the Week 1 Day 1 "Prompting OpenAI Like A Developer" material for an answer to this question!

### Answer

Chain of thought prompting! Let's get the LLM to reason!

In [225]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [227]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Forrest Gump'?")

{'response': 'Forrest Gump is a central character in the film, characterized by his innocence and simplicity. He is portrayed as someone who, despite facing challenges in life, has an impactful and remarkable journey. On page 136, he engages with Jenny, revealing that he is a father, as she explains that their son is named after him. Forrest expresses a sense of amazement and pride over his son, which highlights his nurturing side despite his previous struggles.',
 'context': [('er Lt. Dan. The male\\nnurse sets Lt. Dan down on the rolling bed. Forrest looks up\\nat Lt. Dan as he is wheeled away. A soldier reads the names\\nfrom a pile of letters.\\nSOLDIER", metadata={\'source\': \'data/Forrest_Gump.pdf\', \'page\': 71}), Document(page_content=\'72 .\\nCooper, Larson, Webster, Gump, \\nGump...\\nFORREST\\nI\\\'m Forrest Gump.\\nThe soldier hands Forrest a large pile of letters.\\n"Undeliverable as addressed. No forwarding order on file."\\nJenny Curran Rural Route 2 Greenbow, Ala. 399

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

### Enhancements

I've made three changes to the RAG Application.

1. Allowed it to work with PDF files.
2. Return sources in the response
3. Add converation history

**Changes I made**

- Imported langchain's PyPDFLoader and set the Forrest Gump movie script as the doc that should be spit with our text_splitter
- Augmented the prompt to include domain specific instructions along with the instructions to cite a page number if an answer could be provided
- Include a space for {conversation_history} in our user prompt template so our model would have context around follow up questions

In [ ]:
!pip install langchain_community

In [381]:
### Import langchains PyPDFLoader

from langchain_community.document_loaders import PyPDFLoader

In [382]:
### Load in the PDF - I use the Forrest Gump movie script
### Then split the PDF it into 199 chunks

pdf_doc = PyPDFLoader("data/Forrest_Gump.pdf")
loaded_pdf_doc = str(pdf_doc.load())
doc_chunks = text_splitter.split_texts([loaded_pdf_doc])
len(doc_chunks)

199

In [383]:
### Set up our vectordb with our chunks

vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(doc_chunks))

In [384]:
### Let's test our retreiver

vector_db.search_by_text("Did Forrest serve in the military?", k=3)

### We get some relevant chunks that referrence Forrest and the military or military adjacent information

[('onor.\\nINT. VIETNAM HOSPITAL WARD 81 81\\nForrest steps up to Lt. Dan\'s bed.\\nFORREST\\nGuess what, Lieutenant Dan, they\\nwant to give me a me...\\nForrest stops in mid-sentence as he looks down at the bed. A\\nheavily bandaged soldier with bloodstains lies there. Forrest\\nturns and look at the NURSE.\\nFORREST\\nMa\'am, what\'d they do with \\nLieutenant\\nDan?\\nNURSE\\nThey sent him home.\\nFORREST (V.O.)\\nTwo weeks later, I left Vietnam.\\nINT. BARBER SHOP/GREENBOW - DAY 82 82\\nANCHORMAN\\n(on T.V.)\\nThe ceremony was kicked off with a\\ncandid speech by the President\\nregarding the need for further\\nescalation of the war in Vietnam.", metadata={\'source\': \'data/Forrest_Gump.pdf\', \'page\': 75}), Document(page_content="76 .\\nPresident Johnson awarded four \\nMedals\\nof Honor to men from each of the\\nArmed Services.\\nThe television reveals Forrest as he is awarded the Medal of\\nHonor by President Johnson.\\nPRESIDENT JOHNSON\\nAmerica owes you a debt of \\ngratit

In [410]:
### Updated our RAG_PROMPT_TEMPLATE with context specific info, a clear 'job', and instructions to cite a page number with the response
### Include {conversation_history} as an input variable

RAG_PROMPT_TEMPLATE = """ \
You have been provided with the Forrest Gump movie script and previous conversation history. Your job is to use the provided context and conversation history to answer the user's query. Users may have questions around particular characters, famous quotes, or explanations of certain scenes.

Please provide answers to the user's questions if they pertain to the context or conversation history. If you can provide an answer, make sure to reference the page number that you are referencing to produce your answer. Include this citation at the end of your response in this format: (Page. X).

You may not answer the user's query unless there is specific context in the following text or conversation history.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

Conversation History:
{conversation_history}

User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.conversation_history = []

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        conversation_history_prompt = "\n".join(self.conversation_history)

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(
            user_query=user_query, 
            context=context_prompt,
            conversation_history=conversation_history_prompt
        )
        
        self.conversation_history.append(f"User: {user_query}")
        self.conversation_history.append(f"Assistant: {response}")

        response = self.llm.run([formatted_system_prompt, formatted_user_prompt])

        return {"response": response, "context": context_list}

In [401]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [411]:
### Here we invoke the RAG QA pipeline with an ood question
### We don't get a page number cited since an answer isn't provided

ood_question = retrieval_augmented_qa_pipeline.run_pipeline("What is the tallest building in the world?")

pprint(ood_question['response'])

"I don't know."


In [402]:
### Here we invoke the RAG QA pipeline with an in-domain question
### We get our page number cited since an answer was provided

forrest_best_friend = retrieval_augmented_qa_pipeline.run_pipeline("Did forrest meet a best friend in the military?")

pprint(forrest_best_friend['response'])

('Yes, Forrest Gump met his best friend, Bubba, in the military. Their '
 'friendship developed during their time together in Vietnam, and Bubba was '
 'described as Forrest\'s "best good friend." Bubba had aspirations to become '
 'a shrimp boat captain, but sadly, he died in Vietnam. This friendship is '
 "highlighted in Forrest's reflections on their conversations and experiences "
 'together (Page 69).')


In [403]:
### We ask a follow up question with vague language (e.g. diectic expressions of 'he' and 'him')
### Due to the chat history, the model has context with what this question is about

forrest_promise = retrieval_augmented_qa_pipeline.run_pipeline("And what business did forrest promise he would do with him after the military?")

pprint(forrest_promise['response'])

('Forrest promised Bubba that he would go into the shrimping business with him '
 'after the military. Bubba had a detailed plan for how they could run the '
 'business together, splitting everything fifty-fifty and living on the boat '
 'without paying rent (Page 61).')


In [404]:
### Additional follow up
### We are able to ask if Forrest lived up to "THE" promise that was mentioned earlier

live_up_to_promise = retrieval_augmented_qa_pipeline.run_pipeline("And did he live up to that promise?")

pprint(live_up_to_promise['response'])

('Yes, Forrest lived up to that promise. After the military, he bought a '
 'shrimping boat as he had promised Bubba and became a shrimp boat captain in '
 'his honor. Forrest was committed to making their dream a reality, and he '
 'often reflected on the promise he made to Bubba in Vietnam (Page 95).')


In [407]:
### This is just to show that there was potentially another 'promise' 
### that could've been interpreted in the previous question had there not been chat history

promise_jenny = retrieval_augmented_qa_pipeline.run_pipeline("what promise did Forrest make to Jenny?")

pprint(promise_jenny['response'])

('Forrest promised Jenny that if he was ever in trouble, he should just run '
 'away and not try to be brave. This promise was made during their farewell '
 'conversation, where Jenny expressed her concern for his safety. Forrest '
 'assured her that he would write to her all the time (Page 52).')


In [408]:
### Another follow up on this new 'promise' that was the topic of the current query

promise_jenny_location = retrieval_augmented_qa_pipeline.run_pipeline("Where did they say their farewell?")

pprint(promise_jenny_location['response'])

('Forrest and Jenny said their farewell at a bus stop before Forrest went off '
 'to the military. They had a heartfelt conversation where they made promises '
 'to each other (Page 52).')
